# AI Agent Development

## Pre-requites

### Required Dotnet Packages for AI Agent Development

In [1]:

#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"
#r "nuget: Microsoft.Extensions.AI, 9.10.0"
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.10.0"
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.Identity"
#r "nuget: System.Text.Json, 8.0.0"

Installed Packages Azure.AI.OpenAI, 2.1.0 Azure.Identity, 1.17.0 Microsoft.Agents.AI, 1.0.0-preview.251001.3 Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3 Microsoft.Extensions.AI, 9.10.0 Microsoft.Extensions.AI.Abstractions, 9.10.0 System.Text.Json, 8.0.0

In [2]:
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: Newtonsoft.Json, 13.0.3"

Installed Packages DotNetEnv, 3.1.1 Newtonsoft.Json, 13.0.3

In [3]:
using System;
using System.IO;
using System.ComponentModel;
using System.ClientModel;
using System.Text.Json;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;

using OpenAI;
using OpenAI.Chat;

using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using Newtonsoft.Json;


### Load config

In [4]:
using DotNetEnv;
Env.Load("./.env");

In [5]:
var endpoint = Environment.GetEnvironmentVariable("ENDPOINT") ?? throw new InvalidOperationException("ENDPOINT is required");
var model_id = Environment.GetEnvironmentVariable("MODEL_ID") ?? throw new InvalidOperationException("MODEL_ID is required");
var token = Environment.GetEnvironmentVariable("TOKEN") ?? throw new InvalidOperationException("TOKEN is required");
var uri = new Uri(endpoint);

## Model

### Chat Completion

In [6]:
AzureOpenAIClient azureClient = new(uri, new AzureKeyCredential(token));
ChatClient chatClient = azureClient.GetChatClient(model_id);

var requestOptions = new ChatCompletionOptions()
{
    // MaxCompletionTokens = 13107,
    Temperature = 1.0f,
    TopP = 1.0f,
    FrequencyPenalty = 0.0f,
    PresencePenalty = 0.0f,
    
};

List<OpenAI.Chat.ChatMessage> messages = new List<OpenAI.Chat.ChatMessage>()
{
    new SystemChatMessage("You are a helpful assistant. Should not answer questions outside of travel recommendations."),
    new UserChatMessage("I am going to Paris, what should I see?"),
};

var response = chatClient.CompleteChat(messages, requestOptions);
System.Console.WriteLine(response.Value.Content[0].Text);

Paris is a city rich in history, art, and culture! Some must-see attractions include:

1. **Eiffel Tower** – The symbol of Paris, offering stunning views from its observation decks.
2. **Louvre Museum** – Home to masterpieces like the Mona Lisa and countless other works of art.
3. **Notre-Dame Cathedral** – A stunning example of French Gothic architecture.
4. **Sacré-Cœur Basilica** – Located at Montmartre, offering panoramic city views.
5. **Champs-Élysées & Arc de Triomphe** – A famous avenue leading to the monumental arch.
6. **Musée d'Orsay** – Art museum housed in a former train station, renowned for Impressionist works.
7. **Palace of Versailles** – A short trip outside Paris, famous for its opulent architecture and gardens.
8. **Seine River Cruise** – See many iconic sights from the water.
9. **Le Marais District** – A trendy neighborhood with boutique shops, galleries, and historic sites.
10. **Sainte-Chapelle** – Known for its stunning stained glass windows.

Enjoy your trip a

## Agent

### Agent creation using Microsoft Agent Framework

In [7]:
var agent = new AzureOpenAIClient (uri, new AzureKeyCredential(token))
    .GetChatClient(model_id)
    .CreateAIAgent(
        name: "VacationPlannerAgent",
        description: "An AI Agent that helps customers plan vacations at random destinations",
        instructions: "You are a helpful AI Agent that can help plan vacations for customers at random destinations. Do't allow the user to aks other questions.",
        tools: []
        // AIFunctionFactory.Create((Func<string>)GetRandomDestination)
    );

In [8]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip. result in table format. destination is india "));

| Time         | Activity                                   | Details                                             |
|--------------|--------------------------------------------|-----------------------------------------------------|
| 8:00 AM      | Breakfast at a local restaurant            | Try traditional Indian breakfast options like parathas or idlis. |
| 9:00 AM      | Visit Taj Mahal in Agra                    | Explore the iconic monument and learn its history. |
| 12:00 PM     | Lunch at a nearby restaurant                | Enjoy Indian cuisine such as biryani or dal.       |
| 1:30 PM      | Visit Fatehpur Sikri                        | Tour the UNESCO World Heritage Site and its architectural marvels. |
| 4:00 PM      | Tea break at a local café                   | Relax and enjoy views of the city.                   |
| 5:00 PM      | Shopping at local markets                    | Purchase handicrafts and souvenirs.                 |
| 6:30 PM      | Return to hotel or trav

### Enabling observability for Agents

In [9]:
#r "nuget: OpenTelemetry, 1.13.1"
#r "nuget: OpenTelemetry.Exporter.Console, 1.13.1"

using System;
using OpenTelemetry;
using OpenTelemetry.Trace;
using System.Diagnostics;

Installed Packages OpenTelemetry, 1.13.1 OpenTelemetry.Exporter.Console, 1.13.1

Create a TracerProvider that exports to the console

In [10]:
var tracerProvider = Sdk.CreateTracerProviderBuilder()
    .AddSource("agent-telemetry-source")
    .AddConsoleExporter()
    .Build();

In [11]:
agent = agent
    .AsBuilder()
    .UseOpenTelemetry(sourceName: "agent-telemetry-source")
    .Build();
    

In [12]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip. result in table format. destination is india "));

Certainly! Here's a suggested day trip plan to India, focusing on a popular destination:

| Time          | Activity                                          | Details                                       |
|---------------|---------------------------------------------------|-----------------------------------------------|
| 8:00 AM       | Depart from your location                          | Ensure to start early for maximum enjoyment |
| 11:00 AM      | Arrive in Delhi                                   | Explore India Gate and nearby attractions   |
| 12:00 PM      | Visit Lotus Temple                                | Stunning Bahá'í House of Worship            |
| 1:00 PM       | Lunch at a local restaurant                         | Try Indian cuisine like biryani or curry   |
| 2:30 PM       | Tour Qutub Minar                                | UNESCO World Heritage Site                   |
| 4:00 PM       | Shopping and coffee at Connaught Place            | Enjoy vibrant markets a

### Session-Based Agent

In [13]:
// Create a new thread.
AgentThread thread = agent.GetNewThread();
// Run the agent with the thread.
Console.WriteLine( await agent.RunAsync("Plan me a day trip. result in table format. destination is Dubai.", thread));


| Time          | Activity                                      | Details                                             |
|---------------|----------------------------------------------|-----------------------------------------------------|
| 8:00 AM       | Breakfast at a local café                     | Enjoy traditional Middle Eastern breakfast        |
| 9:00 AM       | Visit Burj Khalifa                            | Admire city views from the observation deck       |
| 11:00 AM      | Explore The Dubai Mall                        | Shopping, aquariums, and indoor ice rink        |
| 1:00 PM       | Lunch at a restaurant within The Dubai Mall | Mediterranean or local cuisine options available |
| 2:30 PM       | Visit Dubai Fountain                          | Witness the spectacular water and light show     |
| 3:30 PM       | Tour of Dubai Museum & Al Fahidi Historic District | Learn about Dubai's history and culture       |
| 5:00 PM       | Relax at Jumeirah Beach                 

### Stateful Agent

In [23]:
// Create a new thread.
AgentThread thread = agent.GetNewThread();
// Run the agent with the thread.
var response = await agent.RunAsync("Hello, how are you?", thread);
Console.WriteLine(response);

// Serialize the thread for storage.
// JsonElement serializedThread = await thread.SerializeAsync();
var serializedThread = JsonConvert.SerializeObject(thread);
// Console.WriteLine(serializedThread);
// Deserialize the thread state after loading from storage.
// AgentThread resumedThread = await agent.DeserializeThreadAsync(serializedThread);
AgentThread resumedThread = JsonConvert.DeserializeObject<ChatClientAgentThread>(serializedThread);

// Run the agent with the resumed thread.
Console.WriteLine( await agent.RunAsync("what is my last question?", resumedThread));

Hello! I'm here to help you plan your perfect vacation. Where would you like to go for your next trip?
I'm here to help you plan your vacation. Please let me know where you'd like to go!


### Agent As tool

In [ ]:
var VacationPlannerAgent = new AzureOpenAIClient (uri, new AzureKeyCredential(token))
    .GetChatClient(model_id)
    .CreateAIAgent(
        name: "VacationPlannerAgent",
        description: "An AI Agent that helps customers plan vacations at random destinations",
        instructions: "You are a helpful AI Agent that can help plan vacations for customers at random destinations. Do't allow the user to aks other questions."
    );

var orchestrator = new AzureOpenAIClient (uri, new AzureKeyCredential(token))
    .GetChatClient(model_id)
    .CreateAIAgent(
        name: "OrchestratorAgent",
        description: "An AI Agent that can delegate tasks to specialized agents",
        instructions: "You are an orchestrator AI Agent that delegates tasks to specialized agents based on user requests.",
        tools: [VacationPlannerAgent.AsAIFunction()]
    );
Console.WriteLine(await orchestrator.RunAsync("I want to plan a vacation to Japan. Can you help me with that?"));


Error: (19,14): error CS1061: 'AIAgent' does not contain a definition for 'Telemetry' and no accessible extension method 'Telemetry' accepting a first argument of type 'AIAgent' could be found (are you missing a using directive or an assembly reference?)

### Expose an agent as an MCP tool

In [15]:
#r "nuget: Microsoft.Extensions.Hosting, 9.0.0"
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
#r "nuget: ModelContextProtocol, 0.4.0-preview.3"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;
using ModelContextProtocol.Server;


Installed Packages microsoft.extensions.dependencyinjection, 9.0.0 Microsoft.Extensions.Hosting, 9.0.0 ModelContextProtocol, 0.4.0-preview.3

Creating MCP tools

In [16]:
McpServerTool tool = McpServerTool.Create(agent.AsAIFunction());
HostApplicationBuilder builder = Host.CreateEmptyApplicationBuilder(settings: null);
builder.Services
    .AddMcpServer()
    .WithStdioServerTransport()
    .WithTools([tool]);

// await builder.Build().RunAsync();